In [2]:
# Import dependencies
import subprocess
import json
import os
from dotenv import load_dotenv
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI
from bit import wif_to_key
from web3 import Web3
from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy
from eth_account import Account

In [3]:
# Load and set environment variables
load_dotenv()
mnemonic=os.getenv("mnemonic")

In [4]:
# Make sure mnemonic loaded
len(mnemonic)

94

In [5]:
# Import constants.py and necessary functions from bit and web3
from constants import *

In [9]:
# Make sure constants laoded
print(BTC)
print(ETH)
print(BTCTEST)

btc
eth
btc-test


In [10]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [16]:
# Create a function called `derive_wallets`
def derive_wallets(mnemonic, coin, numderive):
    command = 'php derive -g --mnemonic="'+str(mnemonic)+'" --numderive="'+str(numderive)+'" --coin="'+str(coin)+'" --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)

In [17]:
# Test derive_wallets function
derive_wallets(mnemonic, ETH, 3)

[{'path': "m/44'/60'/0'/0/0",
  'address': '0x45D09DD827d7E831EF60f4E8EF7EAA9b0b1edC66',
  'xprv': 'xprvA3P4KWwQ2WuHB1vwFMC4nvBXBLoHw2zJLYhtvG4rX1aEf1A83gZDw7eQCamjPSzGF6jbr8KeJ7m3waYKBcNozYN365sja3YXRZ59hb9z38y',
  'xpub': 'xpub6GNQj2UHrtTaPW1QMNj5A48FjNdnLVi9hmdVieUU5M7DXoVGbDsUUuxt3qS3Af3AkTezKgR7oYZgLC217Ffi4gSt7g1E9hZQAMsww1kARe3',
  'privkey': '0xd36eb3f541c90f19705b3b4cfe878cfe3b7456b5ce1e76ace8dfacbccb6333c7',
  'pubkey': '02a2b49f87fc935a35ffc6920dd735044e7f9e76134a321a6ce9c25433dd31f1a8',
  'pubkeyhash': '3a7b0b58dc27f6ee3b1832bfc371abe2e3857672',
  'index': 0},
 {'path': "m/44'/60'/0'/0/1",
  'address': '0xF5a91a5685A147EfC55f6d5549Aa0fc5E37ABF20',
  'xprv': 'xprvA3P4KWwQ2WuHFHjhz1JwNMw6VhgTESnYjsoaeKiCQxRTfDFG1Xfa1n8MUBXo95cBMWk76qdX8oyNUdjet6hDFeFNAMS4mGaPSsoQBNEgb4c',
  'xpub': 'xpub6GNQj2UHrtTaTmpB62qwjVsq3jWwduWQ76jBSi7oyHxSY1aQZ4ypZaSqKTcXijirh1qdV2tL5knykjEzEnfaGUFR8EVUHQHt2mN935n3iR2',
  'privkey': '0xcdd9936584730d93315c064dd59a16ca4ae6ee680b60ac459eb7b7abe88727df',

In [22]:
# Create a dictionary object called coins to store the output from `derive_wallets`.
coins = {'eth':derive_wallets(mnemonic, ETH, 3), 'btc-test':derive_wallets(mnemonic, BTCTEST, 3)}

In [25]:
# Test calling a private key
eth_privatekey = coins['eth'][0]['privkey']
print(eth_privatekey)

0xd36eb3f541c90f19705b3b4cfe878cfe3b7456b5ce1e76ace8dfacbccb6333c7


In [26]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [27]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin, account, to, amount):
    if coin == ETH:
        gasEstimate = w3.eth.estimateGas(
        {"from": account.address, "to": to, "value": amount}
        )
        return {
            "from": account.address,
            "to": to,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address),
        }   
    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])

In [28]:
# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(coin, account, to, amount):
    if coin == ETH:
        tx = create_raw_tx(coin, account, to, amount)
        signed_tx = account.sign_transaction(tx)
        return w3.eth.sendRawTransaction(signed_tx.rawTransaction)
    if coin == BTCTEST:
        raw_tx = create_tx(coin, account, to, amount)
        signed = account.sign_transaction(raw_tx)
        return NetworkAPI.broadcast_tx_testnet(signed)